In [1]:
import numpy as np
import pandas as pd
from time import sleep

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service

In [2]:
driver = webdriver.Chrome(service=Service('Downloads/chromedriver.exe'))

In [3]:
#scroll to bottom of web
def keep_scrolling_to_the_bottom():
    while True:
        previous_scrollY = driver.execute_script( 'return window.scrollY' ) # vị trí hiện tại
        driver.execute_script('window.scrollBy( 0, 130 )') # scroll 1 khoảng 0-130
        sleep(0.4) # time to load
        if previous_scrollY == driver.execute_script( 'return window.scrollY' ): # khi nào không thay đổi thì ngưng
            break

In [22]:
# get link, name, address... information function
def get_link_product(url, num_page = 50):
    all_links = []
    all_names = []
    all_address = []
    all_price = []
    all_sold_num = []
    #get link, name and address
    for page in range(0,50):
    
        url_ = url.format(page) #url
        driver.get(url_) #get url
        sleep(4)
        keep_scrolling_to_the_bottom() # scroll to load data
        
        #get link
        link_elements = driver.find_elements(By.CSS_SELECTOR,'.shopee-search-item-result__item [href]')
        links = [element.get_attribute('href') for element in link_elements]
        all_links += links 
        

        
        for item in range(0, 60):
            
            #get name
            try:
                name_element = driver.find_element('xpath',
                '/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[2]/div[{}]/a/div/div/div[2]/div[1]/div[1]/div'.format(item+1))
                name = name_element.text
                all_names.append(name)
            except NoSuchElementException:
                all_names.append('No Name')
            #get address
            try:
                address_element = driver.find_element('xpath',
                '/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[2]/div[{}]/a/div/div/div[2]/div[4]'.format(item+1))
                address = address_element.text
                all_address.append(address)
            except NoSuchElementException:
                all_names.append('No address')
            
            #get price
            try:
                price_element = driver.find_element('xpath',
                '/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[2]/div[{}]/a/div/div/div[2]/div[2]'.format(item+1))
                price = price_element.text
                all_price.append(price)
            except NoSuchElementException:
                all_price.append(0)
            
            #get sold_num
            try:
                sold_num_element = driver.find_element('xpath',
                '/html/body/div[1]/div/div[2]/div/div/div[2]/div[2]/div[2]/div[{}]/a/div/div/div[2]/div[3]/div[2]'.format(item+1))
                sold_num = sold_num_element.text
                all_sold_num.append(sold_num)
            except NoSuchElementException:
                all_sold_num.append(0)
           
            
    
    return all_links, all_names, all_address,  all_price, all_sold_num 

In [23]:
def df_product(url, df_name, product):
    #create dataframe
    all_links, all_names, all_address, all_price, all_sold_num = get_link_product(url = url,
                                                         num_page = 50)
    df = pd.DataFrame(zip(all_links, all_names, all_address, all_price, all_sold_num),
                      columns = ['Link', 'Name','Address', 'Price','Sold_num'])
    df['Product'] = product
    df.to_csv('Downloads/'+df_name)
    return df

In [24]:
#7 products
list_product = []
products = ['Giày Đá Bóng','Khăn', 'Dép', 'Quần', 'Áo','Mũ','Vòng Tay']
urls = ['https://shopee.vn/search?keyword=gi%C3%A0y%20%C4%91%C3%A1%20b%C3%B3ng&page={}',
        'https://shopee.vn/search?keyword=kh%C4%83n&page={}',
        'https://shopee.vn/search?keyword=d%C3%A9p&page={}',
        'https://shopee.vn/search?keyword=qu%E1%BA%A7n%20t%E1%BB%91t&page={}',
        'https://shopee.vn/search?keyword=%C3%A1o&page={}',
        'https://shopee.vn/search?keyword=m%C5%A9&page={}',
        'https://shopee.vn/search?keyword=v%C3%B2ng%20tay&page={}']
df_names = ['list_giay', 'list_caogot','list_dep','list_quan','list_ao','list_mu','list_vongtay']
for temp in range(0,7):
    list_product.append(df_product(url = urls[temp], df_name = df_names[temp],
                                   product = products[temp]))


In [ ]:
df = pd.concat(list_product)
df.head()

In [ ]:
df.to_csv('Downloads/df_product')